Functors, applicatives and monads in F#
===

If you want an overview in pictures, [Functors, Applicatives And Monads In Pictures](https://adit.io/posts/2013-04-17-functors,_applicatives,_and_monads_in_pictures.html) is a great overview to get introduced to the concept. This post is a translation of that resource into an interactive F# notebook.

Some parts of the source text (especially regarding haskell typeclasses) make no sense in F# context, so we just skip those parts in favor of a more idiomatic F# approach.

Here's a simple value:

In [ ]:
2

2

And we know how to apply a function to this value:

In [ ]:
((+)3) 2

5

Let's extend this by saying that a value can be in a context. For now you can think of a "context" as a box that you can put a value in:

In [ ]:
Some 2 //Value and context

Value
2


Now when you apply a function to this value, you'll get different results **depending on the context**. This is the idea that Functors, Applicatives and Monads are all based on. The *Option* data type is a union type that defines two related contexts: *Some value* and Nothing a.k.a. *None*

In [ ]:

//type Option<'a> = Some 'a | None
let option_cases = [Some 2; None]
option_cases

index,Value,value
0,2,
1,,<null>


In a second we'll see how function application is different when something is *Some a* versus a *None*. First, let's talk about Functors!

Functors
---
When a value is wrapped in a context, you can't apply a normal function to it:

In [ ]:
((+)3) (Some 2)

Error: input.fsx (1,9)-(1,15) typecheck error The type ''a option' does not match the type 'int'
input.fsx (1,3)-(1,4) typecheck error The type ''a option' does not match the type 'int'

This is where *map* comes in. *map* is from the street, *map* knows about contexts. It knows how to apply functions to values that are wrapped in a context. For example, suppose you want to apply (+)3 to Some 2. Use *Option.map*:

In [ ]:
Option.map ((+)3) (Some 2)

Value
5


**Bam!** *Option.map* shows us how it's done! But how does *Option.map* know how to apply the function?

As a small aside, note how the Option.map function is totally unrelated to map functions of other context types (like List.map, Result.map, etc.). This is a somewhat different approach from how Haskell pulls this off with its typeclass support. But let's not diverge, these things are complicated enough as they are 😅

Just what is a Functor, really?
---

A Functor is any data type that defines a *map* function. Here's Option.map's type signature:

In [ ]:
let optionMap (f: ('a -> 'b)) (v : 'a option) : 'b option = Option.map f v

map takes a function (like *plus3*) and a functor (like *Some 2*) and returns a new functor (*Some 5*). And map magically applies this function, because Option is a Functor. It specifies how map applies to Some's and Nothings. This is what the definition of map would look like if we would write it ourselves:

In [ ]:
let optionMap2 f v =
    match v with
    | Some value -> Some (f value)
    | None -> None

Value
5


Here's what is happening behind the scenes when we write `Option.map ((+)3) (Some 2)`:

In [ ]:
Option.map ((+)3) (Some 2)
//1. Unwrap the 2 value from its Some context
//2. Apply the (+)3 function to the value 2, resulting in 5
//3. Rewrap the value 5 back in a Some context 

Value
5


So then you'r like, alright Option.map, please apply ((+)3) to a None?

In [ ]:
Option.map ((+)3) None
//1. the None case contains no value at all
//2. So we don't apply any function
//3. And end up with None

<null>

Like Morpheus in the Matrix, map knows just what to do; you start with Nothing, so you end up with Nothing! map is zen. Now it makes sense why the Option data type exists. For example, here's how you work with a database record in a language without Option:

```
post = Post.find_by_id(1)
if post
  return post.title
else
  return nil
end
```

But in F#:

In [ ]:
type Post = { id : int; title : string }
let findPost id : Post option = Some { id = id; title = "My first post about F#"}
let getPostTitle (p : Post) = p.title

Option.map getPostTitle (findPost 1)

Value
My first post about F#


If findPost returns a post, we will get the title with getPostTitle. If it returns None, we will return None! Pretty neat, huh?

F# code makes frequent use of the pipe operator, so you will often see this instead:

In [ ]:
findPost 1
|> Option.map getPostTitle

Value
My first post about F#


Here's another example: what happens when you apply a function to a list?

In [ ]:
[2;4;6]
|> List.map ((+)3)

index,value
0,5
1,7
2,9


Lists are functors too! Here's an example definition of List.map:

In [ ]:
let rec listMap (f : 'a->'b) (xs: 'a list) : 'b list =
    match xs with
    | [] -> []
    | h :: t -> (f h) :: (listMap f t)

listMap ((+)3) [2;4;6]

index,value
0,5
1,7
2,9


Applicatives
---

Applicatives take it to the next level. With an applicative, our values are wrapped in a context, just like Functors. But our functions are wrapped in a context too!

Yeah. Let that sink in. Applicatives don't kid around. For Applicatives you can define a function called *apply*, which knows how to apply a function *wrapped in a context* to *a value wrapped in a context*:

In [ ]:
module Option =
    let apply ff vv =
        match ff, vv with
        | Some f, Some v -> Some (f v)
        | _ -> None

Some 2
|> Option.apply (Some ((+)3))

Value
5


Using apply can lead to some interesting situations. For example, apply for the List context:

In [ ]:
module List =
    let rec apply (fs : ('a->'b) list) (xs : 'a list) : 'b list =
        [for f in fs do
         for x in xs -> f x]
        
[1;2;3]
|> List.apply [(( * ) 2);(( + )3)]

index,value
0,2
1,4
2,6
3,4
4,5
5,6


Here's something you can do with Applicatives that you can't do with Functors. How do you apply a function that takes two arguments to two wrapped values?

In [ ]:
Option.apply (Option.map (+) (Some 5)) (Some 3)

Value
8


Or using infix operators for readability:

In [ ]:
let (<!>) = Option.map
let (<*>) = Option.apply

(+) <!> Some 5 <*> Some 3

Value
8


Applicative pushes functor aside. "Big boys can use functions with any number of arguments" it says. "Armed with map and apply, I can take any function that expects any number of unwrapped values. Then I pass it all wrapped values, and I get a wrapped value out! Ahahahahah!"

In [ ]:
( * ) <!> Some 5 <*> Some 3

Value
15


Monads
---

How to learn about Monads:

1. Get a PhD in computer science.
2. Throw it away because you don't need it for this section!

Monads add a new twist.

Functors apply a function to a wrapped value:

In [ ]:
((+) 3) <!> Some 2

Value
5


Applicatives apply a wrapped function to a wrapped value:

In [ ]:
(Some ((+) 3)) <*> (Some 2)

Value
5


Monads apply a function **that returns a wrapped value** to an already wrapped value. Monads have a function `bind` to do this.
Let's see an example. Good ol' `Option` is a monad.

Suppose half is a function that only works on even numbers:

In [ ]:
let half (x : int) : int option =
    match x % 2 with
    | 0 -> Some (x / 2)
    | _ -> None

half 666

Value
333


What if we feed it a wrapped value?

In [ ]:
half (Some 666)

Error: input.fsx (1,7)-(1,15) typecheck error This expression was expected to have type
    'int'    
but here has type
    ''a option'    

Ouch! We need to use `Option.bind` to shove our wrapped value into the function. Here's how it works:

In [ ]:
Some 3
|> Option.bind half

<null>

In [ ]:
Some 4
|> Option.bind half

Value
2


In [ ]:
None
|> Option.bind half

<null>

What's happening inside? Here's an example definition of Option's bind:

In [ ]:
let optionBind (f : 'a -> 'b option) (v: 'a option) : 'b option = 
    match v with
    | None -> None
    | Some a -> f a

Some 4
|> optionBind half

Value
2


1. Bind unwraps the value a from its context
2. It feeds the unwrapped value a into the function f
3. The wrapped value (Some 2) comes out

And if you pass in a None it's even simpler:
1. None goes in
2. Nothing has to be done
3. None comes back out again

You can also chain these calls:

In [ ]:
Some 20
|> Option.bind half
|> Option.bind half
|> Option.bind half

<null>

Or with an infix operator if that's more your style:

In [ ]:
let (>>=) v f = Option.bind f v

Some 20 >>= half >>= half >>= half

<null>

Cool stuff! So now we know that `Option` is a Functor, an Applicative and a Monad!

F# also provides us with some syntactical sugar for monads, called computation expressions. After a bit of black magic.



In [ ]:
type OptionBuilder() =
    member this.Return(x) = Some x
    member this.Bind(m, f) = Option.bind f m
    
let option = OptionBuilder()

You can write your Option-aware code like this, which does exactly the same as above:

In [ ]:
option {
    let! twenty = Some 20
    let! h = half twenty
    let! h2 = half h
    let! h3 = half h2
    return h3
}

<null>

Conclusion
---

1. A *Functor* is a data type that implements the `map` function.

In [ ]:
(Some 2) |> Option.map ((+)3)

Value
5


2. An *Applicative* is a data type that implements the `apply` function.

In [ ]:
Some 2 |> Option.apply (Some ((+)3))

Value
5


3. A *Monad* is a data type that implements the `bind` function.

In [ ]:
Some 2 |> Option.bind half

Value
1


4. `Option` implements all three, so it is a functor, an applicative and a monad.

What's the difference between the three?

* **functors**: you apply a function to a wrapped value using `map`
* **applicatives**: you apply a wrapped function to a wrapped value using `apply`
* **monads**: you apply a function that returns a wrapped value (but takes an unwrapped value), to a wrapped value using `bind`